# Weekly project

Today you are going to implement the last parts of the algorithm you started on monday. For reference you can see it below.

![title](algorithm_3.png)

It is a good idea to follow and track the steps in the algorithm in the below implementation. Only take one step at a time.

Once you have the algorithm up and running you can try with a larger dataset to see if your algorithm is able to maintain good accurracy over a longer distance. The larger dataset can be found here:
[Left images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EQu8kmGBDDROtGJ7IkZB2tQBJrxmgY9t8LVM_JuEi83TYw)
[Right images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EcKI_zrXTvpMulizidCZm4oBLJcQ_LTV9Zs6oQFF74JTRQ)

In [1]:
import numpy as np
import cv2 as cv2
import glob

def getK():
    return np.array([[7.188560e+02, 0.000000e+00, 6.071928e+02],
                     [0, 7.188560e+02, 1.852157e+02],
                     [0, 0, 1]])

def getTruePose():
    file = '00.txt'
    return np.genfromtxt(file, delimiter=' ', dtype=None)

<details>
  <summary>tips:</summary>

- [Feature Matching](https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html)
- To get the keypoint of a certain match do: ```kp1[match.queryIdx].pt``` and ```kp2[match.trainIdx].pt```

In [6]:
def extract_keypoints_sift(img1, img2, K, baseline):
    """
    Use SIFT to detect keypoints and compute descriptors in both images,
    and find matches between them using knnMatch with k=2
    """
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    """
    Get the best keypoints by applying the Lowes ratio test
    """
    # LOWES RATIO TEST 
    # each keypoint of the first image is matched with a number of keypoints 
    # from the second image. We keep the 2 best matches for each keypoint 
    # (best matches = the ones with the smallest distance measurement). 
    # Lowe's test checks that the two distances are sufficiently different. 
    # If they are not, then the keypoint is eliminated and will not be used 
    # for further calculations.

    match_points1 = []
    match_points2 = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            match_points1.append(kp1[m.queryIdx].pt)
            match_points2.append(kp2[m.trainIdx].pt)

    
    # Transform best keypoints to nparray
    p1 = np.array(match_points1).astype('float32')
    p2 = np.array(match_points2).astype('float32')

    ##### ############# ##########
    ##### Do Triangulation #######
    ##### ########################
    # project the feature points to 3D with triangulation
    # projection matrix for Left and Right Image
    M_left = K.dot(np.hstack((np.eye(3), np.zeros((3, 1)))))
    M_rght = K.dot(np.hstack((np.eye(3), np.array([[-baseline, 0, 0]]).T)))

    p1_flip = np.vstack((p1.T, np.ones((1, p1.shape[0]))))
    p2_flip = np.vstack((p2.T, np.ones((1, p2.shape[0]))))

    P = cv2.triangulatePoints(M_left, M_rght, p1_flip[:2], p2_flip[:2])

    # Normalize homogeneous coordinates (P->Nx4  [N,4] is the normalizer/scale)
    P = P / P[3]
    land_points = P[:3]

    return land_points.T, p1

In [7]:
def featureTracking(prev_img, next_img, prev_points, world_points):
    """
    Use OpenCV to find the prev_points from the prev_img in the next_img
    Remember to remove points that could not be found from prev_points, next_points, and world_points
    hint: status == 1
    """
    params = dict(winSize=(21, 21),
                 maxLevel=3,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    # from exercise this week 
    next_points, status, error = cv2.calcOpticalFlowPyrLK(prev_img, next_img, prev_points, None, **params)
    # print(status)
    shp = status.shape[0]
    status = status.reshape(shp)
    # print(status)
    prev_points = prev_points[status==1]
    next_points = next_points[status==1]
    world_points = world_points[status==1]

    return world_points, prev_points, next_points


In [10]:
def playImageSequence(l_img_paths, r_img_paths, K):
    baseline = 0.54
    left_img = cv2.imread(l_img_paths[0], 0)
    right_img = cv2.imread(r_img_paths[0], 0)

    ##### ################################# #######
    ##### Get 3D points Using Triangulation #######
    ##### #########################################
    """
    Implement step 1.2 and 1.3
    Store the features in 'reference_2D' and the 3D points (landmarks) in 'landmark_3D'
    hint: use 'extract_keypoints_sift' above
    """
    landmark_3D, reference_2D = extract_keypoints_sift(left_img, right_img, K, baseline)
        

    # reference
    reference_img = left_img

    # Groundtruth for plot
    truePose = getTruePose()
    traj = np.zeros((600, 600, 3), dtype=np.uint8)
    maxError = 0

    for i in range(1, 101):
        print('image: ', i)
        curImage = cv2.imread(l_img_paths[i], 0)
        curImage_R = cv2.imread(r_img_paths[i], 0)

        ##### ############################################################# #######
        ##### Calculate 2D and 3D feature correspndances in t=T-1, and t=T  #######
        ##### #####################################################################
        """
        Implement step 2.2)
        Remember this is a part of a loop, so the initial features are already
        provided in step 1)-1.3) outside the loop in 'reference_2D' and 'landmark_3D'
        """
        
        ##### ####################################### #######
        ##### Calculate relative pose using PNPRansac #######
        ##### ###############################################
        """
        Implement step 2.3)
        """
        landmark_3D, reference_2D, tracked_2Dpoints = featureTracking(reference_img, 
                                                                  curImage, 
                                                                  reference_2D,
                                                                  landmark_3D)
        _, rvec, tvec, inliers = cv2.solvePnPRansac(landmark_3D, reference_2D, K, None)


        ##### ####################################################### #######
        ##### Get Pose and Tranformation Matrix in world coordionates #######
        ##### ###############################################################
        rot, _ = cv2.Rodrigues(rvec)
        tvec = -rot.T.dot(tvec)  # coordinate transformation, from camera to world. What is the XYZ of the camera wrt World
        inv_transform = np.hstack((rot.T, tvec))  # inverse transform. A tranform projecting points from the camera frame to the world frame

        ##### ################################# #######
        ##### Get 3D points Using Triangulation #######
        ##### #########################################
        # re-obtain the 3D points. 
        # hint: use 'extract_keypoints_sift' again
        """
        Implement step 2.4)
        """
        landmark_3D_new, reference_2D_new = extract_keypoints_sift(curImage, curImage_R, K, baseline)
        
        #Project the points from camera to world coordinates
        reference_2D = reference_2D_new.astype('float32')
        landmark_3D = inv_transform.dot(np.vstack((landmark_3D_new.T, np.ones((1, landmark_3D_new.shape[0])))))
        landmark_3D = landmark_3D.T

        ##### ####################### #######
        ##### Done, Next image please #######
        ##### ###############################
        reference_img = curImage

        ##### ################################## #######
        ##### START OF Print and visualize stuff #######
        ##### ##########################################
        # draw images
        draw_x, draw_y = int(tvec[0]) + 300, 600-(int(tvec[2]) + 100)
        true_x, true_y = int(truePose[i][3]) + 300, 600-(int(truePose[i][11]) + 100)

        curError = np.sqrt(
            (tvec[0] - truePose[i][3]) ** 2 +
            (tvec[1] - truePose[i][7]) ** 2 +
            (tvec[2] - truePose[i][11]) ** 2)
        
        if (curError > maxError):
            maxError = curError

        print(tvec[0],tvec[1],tvec[2], rvec[0], rvec[1], rvec[2])
        print([truePose[i][3], truePose[i][7], truePose[i][11]])
        
        text = "Coordinates: x ={0:02f}m y = {1:02f}m z = {2:02f}m".format(float(tvec[0]), float(tvec[1]),
                                                                           float(tvec[2]))
        cv2.circle(traj, (draw_x, draw_y), 1, (0, 0, 255), 2)
        cv2.circle(traj, (true_x, true_y), 1, (255, 0, 0), 2)
        cv2.rectangle(traj, (10, 30), (550, 50), (0, 0, 0), cv2.FILLED)
        cv2.putText(traj, text, (10, 50), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, 8)

        h1, w1 = traj.shape[:2]
        h2, w2 = curImage.shape[:2]
        vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)
        vis[:h1, :w1, :3] = traj
        vis[:h2, w1:w1 + w2, :3] = np.dstack((np.dstack((curImage,curImage)),curImage))

        cv2.imshow("Trajectory", vis)
        k = cv2.waitKey(1) & 0xFF
        if k == 27: break


    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print('Maximum Error: ', maxError)
    ##### ################################ #######
    ##### END OF Print and visualize stuff #######
    ##### ########################################

In [11]:
# Load image paths
left_img_paths = sorted(glob.glob('left/*.png'))
right_img_paths = sorted(glob.glob('right/*.png'))

K = getK()

playImageSequence(left_img_paths, right_img_paths, K)

image:  1
[0.00041157] [-0.01434773] [-6.79091668e-05] [0.00072784] [2.049743e-05] [-0.00016352]
[-0.04690294, -0.02839928, 0.8586941]
image:  2
[-0.00045634] [-0.01738467] [-0.00033289] [0.00073357] [-3.63577141e-05] [-6.7074764e-05]
[-0.09374345, -0.05676064, 1.716275]
image:  3
[-0.00082312] [-0.01936238] [-2.74628098e-05] [0.00066607] [-6.17651492e-05] [3.4987954e-07]
[-0.1406429, -0.08515762, 2.574964]
image:  4
[-0.00147227] [-0.02238842] [-0.00112619] [0.00071765] [-0.00011246] [3.12433361e-05]
[-0.1874858, -0.1135202, 3.432648]
image:  5
[-0.00216886] [-0.02315778] [-0.00123879] [0.00059232] [-0.00015618] [0.00017714]
[-0.2343818, -0.141915, 4.291335]
image:  6
[-0.00256069] [-0.02260855] [-0.00051781] [0.00039273] [-0.00017923] [0.00028504]
[-0.2812195, -0.1702743, 5.148987]
image:  7
[-0.00282761] [-0.02592726] [-0.00106743] [0.00040427] [-0.00020677] [0.00045142]
[-0.3281178, -0.1986703, 6.007777]
image:  8
[-0.0040717] [-0.02296554] [-0.00111374] [7.72642713e-06] [-0.000286

# Challenge 
The current implementation only uses features computed at the current timestep. However, as we process more images we potentially have a lot of features from previous timesteps that are still valid. The challenge is to expand the `extract_keypoints_surf(..., refPoints)` function by giving it old reference points. You should then combine your freshly computed features with the old features and remove all duplicates. This requires you to keep track of old features and 3D points.

Hint 1: The following function `removeDuplicate` can be used for removing duplicates.

Hint 2: you are not interested in points that are behind you, so remember to remove points that are negative in the direction you move.

In [ ]:
def removeDuplicate(queryPoints, refPoints, radius=5):
    # remove duplicate points from new query points,
    for i in range(len(queryPoints)):
        query = queryPoints[i]
        xliml, xlimh = query[0] - radius, query[0] + radius
        yliml, ylimh = query[1] - radius, query[1] + radius
        inside_x_lim_mask = (refPoints[:, 0] > xliml) & (refPoints[:, 0] < xlimh)
        curr_kps_in_x_lim = refPoints[inside_x_lim_mask]

        if curr_kps_in_x_lim.shape[0] != 0:
            inside_y_lim_mask = (curr_kps_in_x_lim[:, 1] > yliml) & (curr_kps_in_x_lim[:, 1] < ylimh)
            curr_kps_in_x_lim_and_y_lim = curr_kps_in_x_lim[inside_y_lim_mask, :]
            if curr_kps_in_x_lim_and_y_lim.shape[0] != 0:
                queryPoints[i] = np.array([0, 0])
    return (queryPoints[:, 0] != 0)